In [1]:
#code to get names of the images in dataset so that lables can be assigned
import pandas as pd
import os

#defining the address of the dataset
DATA_DIR = "F:\\train"
names=[]
i=0
for root, dirs, files in os.walk(DATA_DIR,topdown=False):
    for name in files:
        names.append(name.replace('.jpeg',''))
        #print(names[i])
        #i+=1
print(len(names))
df=pd.read_csv("F:\\train\\trainLabels.csv")
df=df.values.tolist()
#print(sf)

35127


In [13]:
from sklearn.model_selection import train_test_split as tts
import cv2
import numpy as np
#getting all the images and creating the dataset arrays x and y
def getData(start, end, sets):
    
    x=[]
    y=[]
    for i in range (start,end):
        #reading in the image
        img = cv2.imread("F://train//"+df[i][0]+".jpeg")
        #resizing the image
        img = cv2.resize(img,(224,224))
        #changing the array to a numpy n-dimensional array and setting its shape
        img = np.array(img).reshape((224,224,3))
        #adding the image to the training data array
        x.append(img)
        #adding the labels
        y.append(df[i][1])
        i+=1
        if i%1000==0:
            print(i)
    x=np.array(x)
    y=np.array(y)
    y=np.array(y).reshape(len(y),1)
    
    #splitting up the data into training and testing data
    X_train=np.split(x, sets)
    y_train=np.split(y, sets)
    
    return [X_train, y_train]

In [3]:
#normalizaruion function for the dataset
def normalize(x):
    max_x,min_x=x.max(),x.min()
    norm_x=np.zeros(tuple(x.shape))
    norm_img=x.shape[0]
    for nr in range(norm_img):
        norm_x[nr,...]=(x[nr,...]-float(min_x))/(float(max_x-min_x))
    return norm_x

#function for one hot encoding the labels
def one_hot_encode(x):
    from sklearn.preprocessing import OneHotEncoder
    enc=OneHotEncoder(n_values=5)
    ohc_labels=enc.fit_transform(np.array(x).reshape(-1,1)).toarray()
    return ohc_labels

In [4]:
import tensorflow as tf

#declaring tensorflow global variables
x = tf.placeholder("float", shape=[None,224,224,3], name='x')
y = tf.placeholder("float", shape=[None,5], name='y')
keep_probs = tf.placeholder("float", name='keep_probs')

#defining the convolution 2d layer
def conv2d(inp_lyr,ofilters,kernel,stride):
    conv2d_lyr=tf.layers.conv2d(inputs=inp_lyr,
                               filters=ofilters,
                               kernel_size=kernel,
                               strides=stride,
                               padding="same",
                               activation=tf.nn.relu,
                               )
    return conv2d_lyr

#defing the maxpool 2d layer
def maxpool2d(inp_lyr,pool,stride):
    maxp2d=tf.layers.max_pooling2d(inputs=inp_lyr,
                                 pool_size=pool,
                                 strides=stride,
                                 padding="same")
    return maxp2d

#defining the avg pool 2d layer
def avgpool2d(inp_lyr,pool,stride):
    avgp2d=tf.layers.average_pooling2d(inputs=inp_lyr,
                                 pool_size=pool,
                                 strides=stride,
                                 padding="same")
    return avgp2d

D:\Installations\Anaconda3\envs\tfenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def model():
    
    #getting the input
    img_inp=tf.reshape(x, [-1,224,224,3])
    #two convolution and one maxpool layer
    layer=conv2d(img_inp, 64, [7,7], [2,2])
    layer=conv2d(layer,64, [7,7], [2,2])
    layer=maxpool2d(layer, [5,5], [2,2])
    
    #two convolution and one maxpool layer
    layer=conv2d(layer, 128, [5,5], [1,1])
    layer=conv2d(layer, 128, [3,3], [1,1])
    layer=maxpool2d(layer, [3,3], [2,2])
    
    #four convolution and one maxpool layer
    layer=conv2d(layer, 256, [3,3], [1,1])
    layer=conv2d(layer, 256, [3,3], [1,1])
    layer=conv2d(layer, 256, [3,3], [1,1])
    layer=conv2d(layer, 256, [7,7], [2,2])
    layer=maxpool2d(layer, [3,3], [2,2])
    
    #dropout layer
    layer=tf.layers.dropout(layer,0.3)
    
    #four convolution and one maxpool layer
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=maxpool2d(layer, [3,3], [2,2])
    
    #four convolution and one maxpool layer
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=conv2d(layer, 512, [3,3], [1,1])
    layer=avgpool2d(layer, [3,3], [1,1])
    
    #dropout layer
    layer=tf.layers.dropout(layer,0.4)
    
    #flattening layer
    layer=tf.layers.flatten(layer)
    
    #dense (fully connected) layers
    layer=tf.layers.dense(layer, 4096)
    layer=tf.layers.dense(layer, 4096)
    layer=tf.layers.dense(layer, 5)
    
    #activation layer
    layer=tf.nn.softmax(layer)
    
    return layer

In [6]:
#defining the global training variables
cnn_mod=model()
cnn_mod=tf.identity(cnn_mod, name='model')
cross_entropy=-tf.reduce_sum(y*tf.log(cnn_mod))
epoch_batch=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
corr_pred=tf.equal(tf.argmax(cnn_mod, 1), tf.argmax(y, 1))
acc=tf.reduce_mean(tf.cast(corr_pred, tf.float32), name='acc')

In [10]:
#the function which will be called for training the model
def train_model():
    print('Starting...')
    for sets in range (11):
        X_train, y_train = getData(sets*3000, (sets+1)*3000)
        for epoch in range(25):
            for i in range (0,30):
                x_img=normalize(X_train[i])
                y_img=one_hot_encode(y_train[i])
                tr_acc=sess.run(acc,feed_dict={x: x_img, y: y_img, keep_probs: 1.0})
                print('Set:{} Epoch: {} Batch: {} Accuracy = {:.6f}'.format(sets+1, epoch+1, i+1, tr_acc))
                sess.run(epoch_batch,feed_dict={x: x_img, y: y_img, keep_probs: 0.5})

In [11]:
#declaring a session, initializing it and training the model
sess=tf.Session()
sess.run(tf.global_variables_initializer())    
train_model()

Starting...
1000
2000
3000
Set:1 Epoch: 1 Batch: 1 Accuracy = 0.100000
Set:1 Epoch: 1 Batch: 2 Accuracy = 0.760000
Set:1 Epoch: 1 Batch: 3 Accuracy = 0.720000
Set:1 Epoch: 1 Batch: 4 Accuracy = 0.680000
Set:1 Epoch: 1 Batch: 5 Accuracy = 0.700000
Set:1 Epoch: 1 Batch: 6 Accuracy = 0.790000
Set:1 Epoch: 1 Batch: 7 Accuracy = 0.880000
Set:1 Epoch: 1 Batch: 8 Accuracy = 0.700000
Set:1 Epoch: 1 Batch: 9 Accuracy = 0.680000
Set:1 Epoch: 1 Batch: 10 Accuracy = 0.680000
Set:1 Epoch: 1 Batch: 11 Accuracy = 0.810000
Set:1 Epoch: 1 Batch: 12 Accuracy = 0.710000
Set:1 Epoch: 1 Batch: 13 Accuracy = 0.660000
Set:1 Epoch: 1 Batch: 14 Accuracy = 0.750000
Set:1 Epoch: 1 Batch: 15 Accuracy = 0.790000
Set:1 Epoch: 1 Batch: 16 Accuracy = 0.680000
Set:1 Epoch: 1 Batch: 17 Accuracy = 0.730000
Set:1 Epoch: 1 Batch: 18 Accuracy = 0.850000
Set:1 Epoch: 1 Batch: 19 Accuracy = 0.810000
Set:1 Epoch: 1 Batch: 20 Accuracy = 0.740000
Set:1 Epoch: 1 Batch: 21 Accuracy = 0.800000
Set:1 Epoch: 1 Batch: 22 Accuracy = 0

Set:1 Epoch: 7 Batch: 4 Accuracy = 0.680000
Set:1 Epoch: 7 Batch: 5 Accuracy = 0.700000
Set:1 Epoch: 7 Batch: 6 Accuracy = 0.790000
Set:1 Epoch: 7 Batch: 7 Accuracy = 0.880000
Set:1 Epoch: 7 Batch: 8 Accuracy = 0.700000
Set:1 Epoch: 7 Batch: 9 Accuracy = 0.680000
Set:1 Epoch: 7 Batch: 10 Accuracy = 0.680000
Set:1 Epoch: 7 Batch: 11 Accuracy = 0.810000
Set:1 Epoch: 7 Batch: 12 Accuracy = 0.710000
Set:1 Epoch: 7 Batch: 13 Accuracy = 0.660000
Set:1 Epoch: 7 Batch: 14 Accuracy = 0.750000
Set:1 Epoch: 7 Batch: 15 Accuracy = 0.790000
Set:1 Epoch: 7 Batch: 16 Accuracy = 0.680000
Set:1 Epoch: 7 Batch: 17 Accuracy = 0.730000
Set:1 Epoch: 7 Batch: 18 Accuracy = 0.850000
Set:1 Epoch: 7 Batch: 19 Accuracy = 0.810000
Set:1 Epoch: 7 Batch: 20 Accuracy = 0.740000
Set:1 Epoch: 7 Batch: 21 Accuracy = 0.800000
Set:1 Epoch: 7 Batch: 22 Accuracy = 0.770000
Set:1 Epoch: 7 Batch: 23 Accuracy = 0.860000
Set:1 Epoch: 7 Batch: 24 Accuracy = 0.710000
Set:1 Epoch: 7 Batch: 25 Accuracy = 0.730000
Set:1 Epoch: 7 B

Set:1 Epoch: 13 Batch: 6 Accuracy = 0.790000
Set:1 Epoch: 13 Batch: 7 Accuracy = 0.880000
Set:1 Epoch: 13 Batch: 8 Accuracy = 0.700000
Set:1 Epoch: 13 Batch: 9 Accuracy = 0.680000
Set:1 Epoch: 13 Batch: 10 Accuracy = 0.680000
Set:1 Epoch: 13 Batch: 11 Accuracy = 0.810000
Set:1 Epoch: 13 Batch: 12 Accuracy = 0.710000
Set:1 Epoch: 13 Batch: 13 Accuracy = 0.660000
Set:1 Epoch: 13 Batch: 14 Accuracy = 0.750000
Set:1 Epoch: 13 Batch: 15 Accuracy = 0.790000
Set:1 Epoch: 13 Batch: 16 Accuracy = 0.680000
Set:1 Epoch: 13 Batch: 17 Accuracy = 0.730000
Set:1 Epoch: 13 Batch: 18 Accuracy = 0.850000
Set:1 Epoch: 13 Batch: 19 Accuracy = 0.810000
Set:1 Epoch: 13 Batch: 20 Accuracy = 0.740000
Set:1 Epoch: 13 Batch: 21 Accuracy = 0.800000
Set:1 Epoch: 13 Batch: 22 Accuracy = 0.770000
Set:1 Epoch: 13 Batch: 23 Accuracy = 0.860000
Set:1 Epoch: 13 Batch: 24 Accuracy = 0.710000
Set:1 Epoch: 13 Batch: 25 Accuracy = 0.730000
Set:1 Epoch: 13 Batch: 26 Accuracy = 0.690000
Set:1 Epoch: 13 Batch: 27 Accuracy = 0

Set:1 Epoch: 19 Batch: 6 Accuracy = 0.790000
Set:1 Epoch: 19 Batch: 7 Accuracy = 0.880000
Set:1 Epoch: 19 Batch: 8 Accuracy = 0.700000
Set:1 Epoch: 19 Batch: 9 Accuracy = 0.680000
Set:1 Epoch: 19 Batch: 10 Accuracy = 0.680000
Set:1 Epoch: 19 Batch: 11 Accuracy = 0.810000
Set:1 Epoch: 19 Batch: 12 Accuracy = 0.710000
Set:1 Epoch: 19 Batch: 13 Accuracy = 0.660000
Set:1 Epoch: 19 Batch: 14 Accuracy = 0.750000
Set:1 Epoch: 19 Batch: 15 Accuracy = 0.790000
Set:1 Epoch: 19 Batch: 16 Accuracy = 0.680000
Set:1 Epoch: 19 Batch: 17 Accuracy = 0.730000
Set:1 Epoch: 19 Batch: 18 Accuracy = 0.850000
Set:1 Epoch: 19 Batch: 19 Accuracy = 0.810000
Set:1 Epoch: 19 Batch: 20 Accuracy = 0.740000
Set:1 Epoch: 19 Batch: 21 Accuracy = 0.800000
Set:1 Epoch: 19 Batch: 22 Accuracy = 0.770000
Set:1 Epoch: 19 Batch: 23 Accuracy = 0.860000
Set:1 Epoch: 19 Batch: 24 Accuracy = 0.710000
Set:1 Epoch: 19 Batch: 25 Accuracy = 0.730000
Set:1 Epoch: 19 Batch: 26 Accuracy = 0.690000
Set:1 Epoch: 19 Batch: 27 Accuracy = 0

Set:1 Epoch: 25 Batch: 6 Accuracy = 0.790000
Set:1 Epoch: 25 Batch: 7 Accuracy = 0.880000
Set:1 Epoch: 25 Batch: 8 Accuracy = 0.700000
Set:1 Epoch: 25 Batch: 9 Accuracy = 0.680000
Set:1 Epoch: 25 Batch: 10 Accuracy = 0.680000
Set:1 Epoch: 25 Batch: 11 Accuracy = 0.810000
Set:1 Epoch: 25 Batch: 12 Accuracy = 0.710000
Set:1 Epoch: 25 Batch: 13 Accuracy = 0.660000
Set:1 Epoch: 25 Batch: 14 Accuracy = 0.750000
Set:1 Epoch: 25 Batch: 15 Accuracy = 0.790000
Set:1 Epoch: 25 Batch: 16 Accuracy = 0.680000
Set:1 Epoch: 25 Batch: 17 Accuracy = 0.730000
Set:1 Epoch: 25 Batch: 18 Accuracy = 0.850000
Set:1 Epoch: 25 Batch: 19 Accuracy = 0.810000
Set:1 Epoch: 25 Batch: 20 Accuracy = 0.740000
Set:1 Epoch: 25 Batch: 21 Accuracy = 0.800000
Set:1 Epoch: 25 Batch: 22 Accuracy = 0.770000
Set:1 Epoch: 25 Batch: 23 Accuracy = 0.860000
Set:1 Epoch: 25 Batch: 24 Accuracy = 0.710000
Set:1 Epoch: 25 Batch: 25 Accuracy = 0.730000
Set:1 Epoch: 25 Batch: 26 Accuracy = 0.690000
Set:1 Epoch: 25 Batch: 27 Accuracy = 0

...

Set:10 Epoch: 23 Batch: 6 Accuracy = 0.810000
Set:10 Epoch: 23 Batch: 7 Accuracy = 0.780000
Set:10 Epoch: 23 Batch: 8 Accuracy = 0.740000
Set:10 Epoch: 23 Batch: 9 Accuracy = 0.730000
Set:10 Epoch: 23 Batch: 10 Accuracy = 0.730000
Set:10 Epoch: 23 Batch: 11 Accuracy = 0.760000
Set:10 Epoch: 23 Batch: 12 Accuracy = 0.730000
Set:10 Epoch: 23 Batch: 13 Accuracy = 0.730000
Set:10 Epoch: 23 Batch: 14 Accuracy = 0.680000
Set:10 Epoch: 23 Batch: 15 Accuracy = 0.670000
Set:10 Epoch: 23 Batch: 16 Accuracy = 0.680000
Set:10 Epoch: 23 Batch: 17 Accuracy = 0.710000
Set:10 Epoch: 23 Batch: 18 Accuracy = 0.720000
Set:10 Epoch: 23 Batch: 19 Accuracy = 0.720000
Set:10 Epoch: 23 Batch: 20 Accuracy = 0.760000
Set:10 Epoch: 23 Batch: 21 Accuracy = 0.630000
Set:10 Epoch: 23 Batch: 22 Accuracy = 0.660000
Set:10 Epoch: 23 Batch: 23 Accuracy = 0.720000
Set:10 Epoch: 23 Batch: 24 Accuracy = 0.770000
Set:10 Epoch: 23 Batch: 25 Accuracy = 0.810000
Set:10 Epoch: 23 Batch: 26 Accuracy = 0.850000
Set:10 Epoch: 23 

Set:11 Epoch: 4 Batch: 3 Accuracy = 0.790000
Set:11 Epoch: 4 Batch: 4 Accuracy = 0.730000
Set:11 Epoch: 4 Batch: 5 Accuracy = 0.770000
Set:11 Epoch: 4 Batch: 6 Accuracy = 0.760000
Set:11 Epoch: 4 Batch: 7 Accuracy = 0.770000
Set:11 Epoch: 4 Batch: 8 Accuracy = 0.710000
Set:11 Epoch: 4 Batch: 9 Accuracy = 0.770000
Set:11 Epoch: 4 Batch: 10 Accuracy = 0.780000
Set:11 Epoch: 4 Batch: 11 Accuracy = 0.740000
Set:11 Epoch: 4 Batch: 12 Accuracy = 0.710000
Set:11 Epoch: 4 Batch: 13 Accuracy = 0.760000
Set:11 Epoch: 4 Batch: 14 Accuracy = 0.720000
Set:11 Epoch: 4 Batch: 15 Accuracy = 0.840000
Set:11 Epoch: 4 Batch: 16 Accuracy = 0.750000
Set:11 Epoch: 4 Batch: 17 Accuracy = 0.710000
Set:11 Epoch: 4 Batch: 18 Accuracy = 0.720000
Set:11 Epoch: 4 Batch: 19 Accuracy = 0.730000
Set:11 Epoch: 4 Batch: 20 Accuracy = 0.710000
Set:11 Epoch: 4 Batch: 21 Accuracy = 0.810000
Set:11 Epoch: 4 Batch: 22 Accuracy = 0.750000
Set:11 Epoch: 4 Batch: 23 Accuracy = 0.780000
Set:11 Epoch: 4 Batch: 24 Accuracy = 0.79

Set:11 Epoch: 10 Batch: 3 Accuracy = 0.790000
Set:11 Epoch: 10 Batch: 4 Accuracy = 0.730000
Set:11 Epoch: 10 Batch: 5 Accuracy = 0.770000
Set:11 Epoch: 10 Batch: 6 Accuracy = 0.760000
Set:11 Epoch: 10 Batch: 7 Accuracy = 0.770000
Set:11 Epoch: 10 Batch: 8 Accuracy = 0.710000
Set:11 Epoch: 10 Batch: 9 Accuracy = 0.770000
Set:11 Epoch: 10 Batch: 10 Accuracy = 0.780000
Set:11 Epoch: 10 Batch: 11 Accuracy = 0.740000
Set:11 Epoch: 10 Batch: 12 Accuracy = 0.710000
Set:11 Epoch: 10 Batch: 13 Accuracy = 0.760000
Set:11 Epoch: 10 Batch: 14 Accuracy = 0.720000
Set:11 Epoch: 10 Batch: 15 Accuracy = 0.840000
Set:11 Epoch: 10 Batch: 16 Accuracy = 0.750000
Set:11 Epoch: 10 Batch: 17 Accuracy = 0.710000
Set:11 Epoch: 10 Batch: 18 Accuracy = 0.720000
Set:11 Epoch: 10 Batch: 19 Accuracy = 0.730000
Set:11 Epoch: 10 Batch: 20 Accuracy = 0.710000
Set:11 Epoch: 10 Batch: 21 Accuracy = 0.810000
Set:11 Epoch: 10 Batch: 22 Accuracy = 0.750000
Set:11 Epoch: 10 Batch: 23 Accuracy = 0.780000
Set:11 Epoch: 10 Bat

Set:11 Epoch: 15 Batch: 29 Accuracy = 0.640000
Set:11 Epoch: 15 Batch: 30 Accuracy = 0.730000
Set:11 Epoch: 16 Batch: 1 Accuracy = 0.690000
Set:11 Epoch: 16 Batch: 2 Accuracy = 0.660000
Set:11 Epoch: 16 Batch: 3 Accuracy = 0.790000
Set:11 Epoch: 16 Batch: 4 Accuracy = 0.730000
Set:11 Epoch: 16 Batch: 5 Accuracy = 0.770000
Set:11 Epoch: 16 Batch: 6 Accuracy = 0.760000
Set:11 Epoch: 16 Batch: 7 Accuracy = 0.770000
Set:11 Epoch: 16 Batch: 8 Accuracy = 0.710000
Set:11 Epoch: 16 Batch: 9 Accuracy = 0.770000
Set:11 Epoch: 16 Batch: 10 Accuracy = 0.780000
Set:11 Epoch: 16 Batch: 11 Accuracy = 0.740000
Set:11 Epoch: 16 Batch: 12 Accuracy = 0.710000
Set:11 Epoch: 16 Batch: 13 Accuracy = 0.760000
Set:11 Epoch: 16 Batch: 14 Accuracy = 0.720000
Set:11 Epoch: 16 Batch: 15 Accuracy = 0.840000
Set:11 Epoch: 16 Batch: 16 Accuracy = 0.750000
Set:11 Epoch: 16 Batch: 17 Accuracy = 0.710000
Set:11 Epoch: 16 Batch: 18 Accuracy = 0.720000
Set:11 Epoch: 16 Batch: 19 Accuracy = 0.730000
Set:11 Epoch: 16 Batch

Set:11 Epoch: 21 Batch: 25 Accuracy = 0.690000
Set:11 Epoch: 21 Batch: 26 Accuracy = 0.830000
Set:11 Epoch: 21 Batch: 27 Accuracy = 0.730000
Set:11 Epoch: 21 Batch: 28 Accuracy = 0.680000
Set:11 Epoch: 21 Batch: 29 Accuracy = 0.640000
Set:11 Epoch: 21 Batch: 30 Accuracy = 0.730000
Set:11 Epoch: 22 Batch: 1 Accuracy = 0.690000
Set:11 Epoch: 22 Batch: 2 Accuracy = 0.660000
Set:11 Epoch: 22 Batch: 3 Accuracy = 0.790000
Set:11 Epoch: 22 Batch: 4 Accuracy = 0.730000
Set:11 Epoch: 22 Batch: 5 Accuracy = 0.770000
Set:11 Epoch: 22 Batch: 6 Accuracy = 0.760000
Set:11 Epoch: 22 Batch: 7 Accuracy = 0.770000
Set:11 Epoch: 22 Batch: 8 Accuracy = 0.710000
Set:11 Epoch: 22 Batch: 9 Accuracy = 0.770000
Set:11 Epoch: 22 Batch: 10 Accuracy = 0.780000
Set:11 Epoch: 22 Batch: 11 Accuracy = 0.740000
Set:11 Epoch: 22 Batch: 12 Accuracy = 0.710000
Set:11 Epoch: 22 Batch: 13 Accuracy = 0.760000
Set:11 Epoch: 22 Batch: 14 Accuracy = 0.720000
Set:11 Epoch: 22 Batch: 15 Accuracy = 0.840000
Set:11 Epoch: 22 Batch

In [12]:
#saving the model so that it can be read back later on
save_model_path='F:/Work/capstone'
saver = tf.train.Saver()
save_path = saver.save(sess, save_model_path)

In [15]:
#preparing the test data
X_test, y_test = getData(33000, 35000, 20)

34000
35000


In [17]:
#defining the function that will test the model on unknown data
def test_model():
    
    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_probs:0')
        loaded_acc = loaded_graph.get_tensor_by_name('acc:0')
        
        # Get accuracy in batches for memory limitations
        test_batch_acc_total = 0
        test_batch_count = 0
        
        for i in range(0, 20):
            x_test_img=normalize(X_test[i])
            y_test_img=one_hot_encode(y_test[i])
            test_batch_acc_total += sess.run(
                loaded_acc,
                feed_dict={loaded_x: x_test_img, loaded_y: y_test_img, loaded_keep_prob: 1.0})
            test_batch_count += 1

        print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))


test_model()

INFO:tensorflow:Restoring parameters from F:/Work/capstone
Testing Accuracy: 0.7530000030994415

